## Code for shortest path between two pixel locations.

## Given as input:
1. Image Segment (Enter as a Matrix)
2. Pixel Locations p and q.
3. Predefined set V of intensity values.
4. The Path Type (4-/8-/m-) 



In [1]:
import itertools
import numpy as np
import heapq as hq
import time

In [2]:
def to_index(x,y,img):
    return x * img.shape[1] + y

In [3]:
def to_coordinates(index,img):
    return index // img.shape[1], index % img.shape[1]

In [40]:
def create_adjacency(type_con,my_list,img):
    directions_8     = list(itertools.product([0, 1, -1], [0, 1, -1]))
    adjacency = np.array(np.ones((36,36)) * np.inf)
    if(type_con == "8_conn"):
        direct = directions_8
    else:
        direct = [(i,j) for (i,j) in directions_8 if abs(i)+abs(j) != 2 ]             
        direct_diag = [(i,j) for (i,j) in directions_8 if abs(i)+abs(j) == 2 ]
    
    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            if img[i,j] not in my_list:
                continue
            flag = 0
            #print(i,j,img[i,j])
            for y_diff, x_diff in direct:
                #print(y_diff, x_diff,img[i + y_diff,j + x_diff])
                if (img[i + y_diff, j + x_diff] in my_list) and \
                                                (abs(y_diff) + abs(x_diff) != 0):
                    adjacency[to_index(i, j,img),to_index \
                                                (i + y_diff, j + x_diff,img)] = 1
                    #print(i + y_diff, j + x_diff)
            if(type_con == "m_conn"):
                for y_diff, x_diff in direct_diag:
                   # print(y_diff, x_diff,img[i + y_diff,j + x_diff])
                    if (img[i + y_diff, j + x_diff] in my_list) and \
                                    (img[i,j+x_diff] not in my_list) and \
                                            (img[i+y_diff,j] not in my_list) and \
                                                      (abs(y_diff) + abs(x_diff) != 0):
                        adjacency[to_index(i, j,img),to_index\
                                                      (i + y_diff, j + x_diff,img)] = 1
                        #print(i + y_diff, j + x_diff)
    return adjacency

In [5]:
def dijkstra_code(G, source,target):
    inf = float('Inf')
    n = len(G)
    prev = [[] for i in range(n)]
    Q = [(0, source)]
    d = [inf for i in range(n)]
    d[source]= 0
    while len(Q)!=0:
        (cost, u) = hq.heappop(Q)
        #print("POP",cost,u,Q)
        for v in range(n):
            if d[v]  > d[u] + G[u][v]:
                d[v] = d[u] + G[u][v]
                hq.heappush(Q, (d[v], v))
                prev[v].append(u) 
                #print("PUSH",v,u,Q)
    return d[target],prev

In [6]:
def print_path(prev,target,source,img):
    path = []
    path.append((to_coordinates(target,img)))
    a = prev[target][0]
    while a != source:
        path.append((to_coordinates(a,img)))
        a = prev[a][0]
    path.append((to_coordinates(source,img)))
    path = path[::-1]
    print("Path Found",end = " ")
    for p in path:
        print("-->",p,end = " ")
    print("\n")

In [7]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector

In [41]:
def user_input():
    try: 
        n = int(input("Enter the number of rows in a matrix: "))
        print("Enter elements row-wise")
        img_original = [[int(i) for i in input().split()] for i in range(n)]
        img_original = np.array(img_original)
        my_list      = [int(i) for i in input('Enter The Predefined List eg. 0 1:')\
                                                                            .split()] 
        p_coord      = [int(i) for i in input('Enter The Source Coordinates \
                                        (Note: rows/cols start with 1 not 0) eg. 4 1 :').split()] 
        q_coord      = [int(i) for i in input('Enter The Target Coordinates \
                                        (Note: rows/cols start with 1 not 0) eg. 1 4 :').split()] 
        #type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
    except:
        print("Incorrect Values Inputed")
    return img_original, my_list, p_coord, q_coord

In [17]:
def compute_result(img_original, my_list, p_coord, q_coord):
    start_time = time.time()
    img = np.pad(np.array(img_original), 1, pad_with,padder = 999)
    source = to_index(p_coord[0], p_coord[1],img)
    target = to_index(q_coord[0], q_coord[1],img)
    adjacency  = create_adjacency(type_con,my_list,img)
    pathlength,prev = dijkstra_code(adjacency,source,target)
    if (pathlength != np.inf):
        print("Path length found to be: %d" %(pathlength))
        print_path(prev,target,source,img)
    else:
        print("No Path Found")
    print("--- Runtime for Code: %s seconds ---" % (time.time() - start_time))

## Problem 2 - a - i ( 4 - Path )

In [18]:
img_original, my_list, p_coord, q_coord = user_input()
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter the number of rows in a matrix: 4
Enter elements row-wise
3 1 2 1
2 2 0 2
1 2 1 1
1 0 1 2
Enter The Predefined List eg. 0 1:0 1
Enter The Source Coordinates (Note: rows/cols start with 1 not 0) eg. 4 1 :4 1
Enter The Target Coordinates (Note: rows/cols start with 1 not 0) eg. 1 4 :1 4
Enter The Path Type (4_conn/8_conn/m_conn):4_conn
No Path Found
--- Runtime for Code: 0.001989126205444336 seconds ---


## Problem 2 - a - ii  ( 8 - Path )

In [19]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):8_conn
Path length found to be: 4
Path Found --> (4, 1) --> (4, 2) --> (3, 3) --> (2, 3) --> (1, 4) 

--- Runtime for Code: 0.0009963512420654297 seconds ---


## Problem 2 - a - iii  ( m - Path )

In [20]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):m_conn
Path length found to be: 5
Path Found --> (4, 1) --> (4, 2) --> (4, 3) --> (3, 3) --> (2, 3) --> (1, 4) 

--- Runtime for Code: 0.001986980438232422 seconds ---


## Problem 2 - b - i ( 4 - Path )

In [21]:
img_original, my_list, p_coord, q_coord = user_input()
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter the number of rows in a matrix: 4
Enter elements row-wise
3 1 2 1
2  2 0 2
1 2 1 1 
1 0 1 2
Enter The Predefined List eg. 0 1:1 2 
Enter The Source Coordinates (Note: rows/cols start with 1 not 0) eg. 4 1 :4 1
Enter The Target Coordinates (Note: rows/cols start with 1 not 0) eg. 1 4 :1 4
Enter The Path Type (4_conn/8_conn/m_conn):4_conn
Path length found to be: 6
Path Found --> (4, 1) --> (3, 1) --> (2, 1) --> (2, 2) --> (1, 2) --> (1, 3) --> (1, 4) 

--- Runtime for Code: 0.0009965896606445312 seconds ---


## Problem 2 - b - ii ( 8 - Path )

In [22]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):8_conn
Path length found to be: 4
Path Found --> (4, 1) --> (3, 1) --> (2, 2) --> (1, 3) --> (1, 4) 

--- Runtime for Code: 0.0009849071502685547 seconds ---


## Problem 2 - b - iii ( m - Path )

In [23]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):m_conn
Path length found to be: 6
Path Found --> (4, 1) --> (3, 1) --> (2, 1) --> (2, 2) --> (1, 2) --> (1, 3) --> (1, 4) 

--- Runtime for Code: 0.00099945068359375 seconds ---


## Additional Example 1

In [27]:
img_original, my_list, p_coord, q_coord = user_input()
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter the number of rows in a matrix: 3
Enter elements row-wise
0 1 0
0 1 0
0 0 1
Enter The Predefined List eg. 0 1:1
Enter The Source Coordinates (Note: rows/cols start with 1 not 0) eg. 4 1 :3 3
Enter The Target Coordinates (Note: rows/cols start with 1 not 0) eg. 1 4 :1 2
Enter The Path Type (4_conn/8_conn/m_conn):4_conn
No Path Found
--- Runtime for Code: 0.0009992122650146484 seconds ---


In [28]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):8_conn
Path length found to be: 2
Path Found --> (3, 3) --> (2, 2) --> (1, 2) 

--- Runtime for Code: 0.000995635986328125 seconds ---


In [29]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):m_conn
Path length found to be: 2
Path Found --> (3, 3) --> (2, 2) --> (1, 2) 

--- Runtime for Code: 0.0009961128234863281 seconds ---


## Additional Example 2

In [30]:
img_original, my_list, p_coord, q_coord = user_input()
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter the number of rows in a matrix: 4
Enter elements row-wise
0 1 1 0
1 0 2 0
1 2 0 1
0 0 1 0
Enter The Predefined List eg. 0 1:1 2
Enter The Source Coordinates (Note: rows/cols start with 1 not 0) eg. 4 1 :4 3
Enter The Target Coordinates (Note: rows/cols start with 1 not 0) eg. 1 4 :1 2
Enter The Path Type (4_conn/8_conn/m_conn):4_conn
No Path Found
--- Runtime for Code: 0.000997304916381836 seconds ---


In [31]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):8_conn
Path length found to be: 3
Path Found --> (4, 3) --> (3, 2) --> (2, 1) --> (1, 2) 

--- Runtime for Code: 0.0009992122650146484 seconds ---


In [32]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):m_conn
Path length found to be: 4
Path Found --> (4, 3) --> (3, 2) --> (2, 3) --> (1, 3) --> (1, 2) 

--- Runtime for Code: 0.0019910335540771484 seconds ---


## Additional Example 3

In [33]:
img_original, my_list, p_coord, q_coord = user_input()
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter the number of rows in a matrix: 3
Enter elements row-wise
1 1 0
0 1 1
1 0 1
Enter The Predefined List eg. 0 1:1
Enter The Source Coordinates (Note: rows/cols start with 1 not 0) eg. 4 1 :3 3
Enter The Target Coordinates (Note: rows/cols start with 1 not 0) eg. 1 4 :1 1
Enter The Path Type (4_conn/8_conn/m_conn):4_conn
Path length found to be: 4
Path Found --> (3, 3) --> (2, 3) --> (2, 2) --> (1, 2) --> (1, 1) 

--- Runtime for Code: 0.0009963512420654297 seconds ---


In [34]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):8_conn
Path length found to be: 2
Path Found --> (3, 3) --> (2, 2) --> (1, 1) 

--- Runtime for Code: 0.0009968280792236328 seconds ---


In [35]:
try:
    type_con      = input('Enter The Path Type (4_conn/8_conn/m_conn):')
except:
    print("Incorrect Values Inputed")
compute_result(img_original, my_list, p_coord, q_coord)

Enter The Path Type (4_conn/8_conn/m_conn):m_conn
Path length found to be: 4
Path Found --> (3, 3) --> (2, 3) --> (2, 2) --> (1, 2) --> (1, 1) 

--- Runtime for Code: 0.001991748809814453 seconds ---
